# STEP 1 — Install Libraries

In [ ]:
!pip install requests pandas numpy scikit-learn xgboost matplotlib gradio


STEP 2 — Fetch Real-Time Weather Data (Open-Meteo API)

In [ ]:
import requests
import pandas as pd

def get_weather(lat, lon):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}&hourly=temperature_2m,relativehumidity_2m,"
        f"pressure_msl,rain,wind_speed_10m&past_days=1&forecast_days=1"
    )
    response = requests.get(url).json()

    # Latest (last hour) data
    idx = -1
    data = {
        "temp": response["hourly"]["temperature_2m"][idx],
        "humidity": response["hourly"]["relativehumidity_2m"][idx],
        "pressure": response["hourly"]["pressure_msl"][idx],
        "rain": response["hourly"]["rain"][idx],
        "wind_speed": response["hourly"]["wind_speed_10m"][idx],
    }
    return data

# Test
get_weather(12.97, 77.59)   # Bengaluru


{'temp': 16.0,
 'humidity': 91,
 'pressure': 1014.5,
 'rain': 0.0,
 'wind_speed': 6.9}

STEP 3 — Fetch Real-Time Water Level (NOAA Free API)

In [ ]:
def get_water_level(station_id="9414290"):  # Default station
    url = (
        f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"
        f"product=water_level&application=web_services&"
        f"date=latest&datum=MLLW&station={station_id}&time_zone=gmt&"
        f"units=metric&format=json"
    )

    response = requests.get(url).json()

    try:
        return {"water_level": float(response["data"][0]["v"])}
    except:
        return {"water_level": None}

# Test
get_water_level()


{'water_level': 0.163}

STEP 4 — Combine Real-Time Weather + Water Level

In [ ]:
def get_realtime_data(lat, lon, station="9414290"):
    weather = get_weather(lat, lon)
    water   = get_water_level(station)

    combined = {**weather, **water}
    return pd.DataFrame([combined])


STEP 5 — Generate Historical Training Dataset

In [ ]:
def get_historical_weather(lat, lon, start="2024-01-01", end="2024-01-31"):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={start}&end_date={end}&"
        f"hourly=temperature_2m,relativehumidity_2m,pressure_msl,rain,wind_speed_10m"
    )
    response = requests.get(url).json()

    df = pd.DataFrame({
        "date": response["hourly"]["time"],
        "temp": response["hourly"]["temperature_2m"],
        "humidity": response["hourly"]["relativehumidity_2m"],
        "pressure": response["hourly"]["pressure_msl"],
        "rain": response["hourly"]["rain"],
        "wind_speed": response["hourly"]["wind_speed_10m"]
    })
    df["date"] = pd.to_datetime(df["date"])
    return df

weather_df = get_historical_weather(12.97, 77.59, start="2025-10-01", end="2025-10-31")
weather_df.head()

,date,temp,humidity,pressure,rain,wind_speed
0,2025-10-01 00:00:00,19.5,94,1009.5,0.0,9.7
1,2025-10-01 01:00:00,19.6,93,1009.9,0.0,11.3
2,2025-10-01 02:00:00,20.8,86,1010.7,0.0,11.7
3,2025-10-01 03:00:00,22.8,73,1011.0,0.0,12.6
4,2025-10-01 04:00:00,24.8,63,1010.4,0.0,16.4


STEP 6 — Get NOAA Historical Water Levels

In [ ]:
import datetime

def get_noaa_historical(station="9414290", start=None, end=None):
    if start is None:
        # Default to 3 days ago
        start = (datetime.date.today() - datetime.timedelta(days=3)).strftime("%Y%m%d")
    if end is None:
        # Default to today
        end = datetime.date.today().strftime("%Y%m%d")

    url = (
        f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"
        f"product=water_level&datum=MLLW&station={station}&"
        f"begin_date={start}&end_date={end}&units=metric&format=json&time_zone=gmt"
    )

    response = requests.get(url).json()

    if "error" in response:
        print(f"NOAA API Error: {response['error']['message']}")
        return pd.DataFrame()  # Return empty DataFrame on API error
    elif "data" not in response:
        print(f"Unexpected NOAA API response format: 'data' key missing. Response: {response}")
        return pd.DataFrame()  # Return empty DataFrame if 'data' is unexpectedly missing

    df = pd.DataFrame(response["data"])
    df["date"] = pd.to_datetime(df["t"])
    df["water_level"] = pd.to_numeric(df["v"], errors="coerce")

    return df[["date", "water_level"]]

# Aligning the date range with weather_df
noaa_start_date = "20251001"
noaa_end_date = "20251031"
water_df = get_noaa_historical(start=noaa_start_date, end=noaa_end_date)
water_df.head()

,date,water_level
0,2025-10-01 00:00:00,1.580
1,2025-10-01 00:06:00,1.597
2,2025-10-01 00:12:00,1.604
3,2025-10-01 00:18:00,1.598
4,2025-10-01 00:24:00,1.596


STEP 7 — Merge Weather + Water Level + Create Flood Labels

In [ ]:
# Align water_df dates to the nearest hour for merging with hourly weather_df
water_df['date'] = water_df['date'].dt.floor('H')

df = weather_df.merge(water_df, on="date", how="inner")

# Create flood label (0 = low risk, 1 = high risk)
df["flood_risk"] = (
    (df["rain"] > 20) |
    (df["water_level"] > 1.5)
).astype(int)

df.head()

/tmp/ipython-input-1303275135.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  water_df['date'] = water_df['date'].dt.floor('H')


,date,temp,humidity,pressure,rain,wind_speed,water_level,flood_risk
0,2025-10-01,19.5,94,1009.5,0.0,9.7,1.580,1
1,2025-10-01,19.5,94,1009.5,0.0,9.7,1.597,1
2,2025-10-01,19.5,94,1009.5,0.0,9.7,1.604,1
3,2025-10-01,19.5,94,1009.5,0.0,9.7,1.598,1
4,2025-10-01,19.5,94,1009.5,0.0,9.7,1.596,1


STEP 8 — Train ML Model (XGBoost)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

features = ["temp", "humidity", "pressure", "rain", "wind_speed", "water_level"]
X = df[features]
y = df["flood_risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = XGBClassifier(n_estimators=300, max_depth=6)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1145
           1       1.00      1.00      1.00       343

    accuracy                           1.00      1488
   macro avg       1.00      1.00      1.00      1488
weighted avg       1.00      1.00      1.00      1488



STEP 9 — Advanced Gradio UI

In [ ]:
import gradio as gr
import datetime

def predict_flood():
    # Fetch real-time data
    lat, lon = 12.97, 77.59
    realtime = get_realtime_data(lat, lon)

    prediction = model.predict(realtime)[0]
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Beautify data view
    data_html = "<br>".join([
        f"<b>Temperature:</b> {realtime['temp'].values[0]} °C",
        f"<b>Humidity:</b> {realtime['humidity'].values[0]} %",
        f"<b>Pressure:</b> {realtime['pressure'].values[0]} hPa",
        f"<b>Rainfall:</b> {realtime['rain'].values[0]} mm",
        f"<b>Wind Speed:</b> {realtime['wind_speed'].values[0]} m/s",
        f"<b>Water Level:</b> {realtime['water_level'].values[0]} m",
    ])

    # Animated prediction messages
    if prediction == 1:
        flood_status = f"""
        <div style='background:#ff4d4d;padding:20px;border-radius:10px;
        animation: pulse 1s infinite;text-align:center;color:white;font-size:22px;font-weight:bold;'>
            ⚠️ HIGH FLOOD RISK ⚠️<br>
            <span style='font-size:18px;'>Immediate caution required</span>
        </div>

        <style>
        @keyframes pulse {{
            0% {{ transform: scale(1); }}
            50% {{ transform: scale(1.05); }}
            100% {{ transform: scale(1); }}
        }}
        </style>
        """
    else:
        flood_status = f"""
        <div style='background:#4CAF50;padding:20px;border-radius:10px;
        animation: glow 2s infinite;text-align:center;color:white;font-size:22px;font-weight:bold;'>
            🟢 LOW FLOOD RISK 🟢<br>
            <span style='font-size:18px;'>Area is safe currently</span>
        </div>

        <style>
        @keyframes glow {{
            0% {{ box-shadow: 0 0 5px #4CAF50; }}
            50% {{ box-shadow: 0 0 20px #4CAF50; }}
            100% {{ box-shadow: 0 0 5px #4CAF50; }}
        }}
        </style>
        """

    # Combine everything in UI
    final_output = f"""
    <h2 style='text-align:center;'>🌧️ Real-Time Flood Prediction</h2>
    <p style='text-align:center;font-size:16px;color:#555;'>Last Updated: <b>{timestamp}</b></p>

    {flood_status}

    <div style='margin-top:20px;padding:20px;border:2px solid #ddd;border-radius:10px;
    background:#f9f9f9;font-size:18px;'>
        <h3 style='text-align:center;'>📊 Live Environmental Data</h3>
        <p style='font-size:17px;'>{data_html}</p>
    </div>
    """

    return final_output


ui = gr.Interface(
    fn=predict_flood,
    inputs=[],
    outputs=gr.HTML(),
    title="🌊 AI-Powered Real-Time Flood Prediction System",
    description="Automatically fetches live weather & tidal data and predicts flood risk with animations."
)

ui.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://205fc5e75a72607b5a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Real time data stored as csv

In [ ]:
import requests
import pandas as pd
import os
from datetime import datetime
import pytz

LAT = 12.9716
LON = 77.5946
CSV_FILE = "open_meteo_live_data.csv"

def fetch_open_meteo():
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={LAT}&longitude={LON}&current=temperature_2m,"
        f"relative_humidity_2m,precipitation,rain,pressure_msl,wind_speed_10m"
    )
    return requests.get(url).json()


def save_open_meteo_to_csv():
    data = fetch_open_meteo()["current"]

    tz = pytz.timezone("Asia/Kolkata")
    timestamp = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

    row = {
        "timestamp": timestamp,
        "temperature": data["temperature_2m"],
        "humidity": data["relative_humidity_2m"],
        "rain": data["rain"],
        "precipitation": data["precipitation"],
        "pressure": data["pressure_msl"],
        "wind_speed": data["wind_speed_10m"]
    }

    # If file does NOT exist → create new CSV
    if not os.path.exists(CSV_FILE):
        df = pd.DataFrame([row])
        df.to_csv(CSV_FILE, index=False)
        return

    # Otherwise → append
    df = pd.read_csv(CSV_FILE)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    # Keep only the latest 50 rows
    if len(df) > 50:
        df = df.tail(50)

    df.to_csv(CSV_FILE, index=False)


# Test once
save_open_meteo_to_csv()
print("Open-Meteo data saved!")


Open-Meteo data saved!


**Ensuring fetching of real time data**

In [ ]:
import time

while True:
    save_open_meteo_to_csv()
    print("Saved both sources...")
    time.sleep(20)   # every 20 seconds


Saved both sources...


KeyboardInterrupt: 